In [ ]:
import numpy as np
import scipy.special as sp

from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import copy
import time
import random

import torch
import torch.nn as nn
import torch.optim as optim
import collections
from scipy.io import loadmat


In [ ]:
model_type = "conv"

**Introduce experience replay.**

In [ ]:
Transition = collections.namedtuple('Experience',
                                    field_names=['state', 'action',
                                                 'next_state', 'reward',
                                                 'is_game_on'])

class ExperienceReplay:
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = collections.deque(maxlen=capacity)

    def __len__(self):
        return len(self.memory)

    def push(self, transition):
        self.memory.append(transition)

    def sample(self, batch_size, device = 'cuda:0'):
        indices = np.random.choice(len(self.memory), batch_size, replace = False)
        
        states, actions, next_states, rewards, isgameon = zip(*[self.memory[idx] 
                                                                for idx in indices])
        
        return torch.Tensor(states).type(torch.float).to(device), \
               torch.Tensor(actions).type(torch.long).to(device), \
               torch.Tensor(next_states).to(device), \
               torch.Tensor(rewards).to(device), torch.tensor(isgameon).to(device)

**Networks definition.**

In [ ]:
class fc_nn(nn.Module):
    def __init__(self, Ni, Nh1, Nh2, No = 4):
        super().__init__()
        
        self.fc1 = nn.Linear(Ni, Nh1)
        self.fc2 = nn.Linear(Nh1, Nh2)
        self.fc3 = nn.Linear(Nh2, No)
        
        self.act = nn.ReLU()
        
    def forward(self, x, classification = False, additional_out=False):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        out = self.fc3(x)
        
        return out

In [ ]:
class conv_nn(nn.Module):
    
    channels = [16, 32, 64]
    kernels = [3, 3, 3]
    strides = [1, 1, 1]
    in_channels = 1
    
    def __init__(self, rows, cols, n_act):
        super().__init__()
        self.rows = rows
        self.cols = cols

        self.conv = nn.Sequential(nn.Conv2d(in_channels = self.in_channels,
                                            out_channels = self.channels[0],
                                            kernel_size = self.kernels[0],
                                            stride = self.strides[0]),
                                  nn.ReLU(),
                                  nn.Conv2d(in_channels = self.channels[0],
                                            out_channels = self.channels[1],
                                            kernel_size = self.kernels[1],
                                            stride = self.strides[1]),
                                  nn.ReLU()
                                 )
        
        size_out_conv = self.get_conv_size(rows, cols)
        
        self.linear = nn.Sequential(nn.Linear(size_out_conv, rows*cols*2),
                                    nn.ReLU(),
                                    nn.Linear(rows*cols*2, int(rows*cols/2)),
                                    nn.ReLU(),
                                    nn.Linear(int(rows*cols/2), n_act),
                                   )

    def forward(self, x):
        x = x.view(len(x), self.in_channels, self.rows, self.cols)
        out_conv = self.conv(x).view(len(x),-1)
        out_lin = self.linear(out_conv)
        return out_lin
    
    def get_conv_size(self, x, y):
        out_conv = self.conv(torch.zeros(1,self.in_channels, x, y))
        return int(np.prod(out_conv.size()))

In [ ]:
def DoubleQloss(batch, Q1_net, Q2_net, gamma=0.95, device="cuda"):
    states, actions, next_states, rewards, _ = batch
    lbatch = len(states)
    
    # Prepare the data
    states = states.view(lbatch, -1).to(device)
    actions = actions.to(device)
    next_states = next_states.view(lbatch, -1).to(device)
    rewards = rewards.to(device)
    
    # Current Q values based on the actions taken
    state_action_values = Q1_net(states).gather(1, actions.unsqueeze(-1)).squeeze(-1)
    
    # Action selection by Q1_net and value evaluation by Q2_net
    # Select the best action in next state according to Q1_net
    next_actions = Q1_net(next_states).max(1)[1].unsqueeze(-1)
    # Evaluate these actions using Q2_net
    next_state_values = Q2_net(next_states).gather(1, next_actions).squeeze(-1)
    
    # Detach next state values from the graph to prevent gradients from flowing
    next_state_values = next_state_values.detach()
    
    # Calculate expected Q values for the current states
    expected_state_action_values = (next_state_values * gamma) + rewards
    
    # Compute loss
    loss = nn.MSELoss()(state_action_values, expected_state_action_values)
    
    return loss


**Import the maze and define the environment.**

In [ ]:
from scipy.io import loadmat

from environment import MazeEnvironment
maze_data = loadmat('Maze_DQN.mat')  # Ensure this matches the structure of the .mat file
maze = maze_data['Maze']

initial_position = [16,4]
goal = [3, 13]

maze_env = MazeEnvironment(maze, initial_position, goal)

maze_env.draw_current()

**Define the agent and the buffer for experience replay.**

In [ ]:
buffer_capacity = 10000
buffer_start_size = 1000
memory_buffer = ExperienceReplay(buffer_capacity)

In [ ]:
from agent import Agent
agent = Agent(maze = maze_env,
              memory_buffer = memory_buffer,
              use_softmax = False
             )

** Define the network.**

In [ ]:
# Modified code for Double Q-Learning
if model_type == "conv":
    Q1_net = conv_nn(len(maze), len(maze), 4)
    Q2_net = conv_nn(len(maze), len(maze), 4)
    averaged_net =  conv_nn(len(maze), len(maze), 4)
else:
    Q1_net= fc_nn(maze.size, maze.size, maze.size, 4)
    Q2_net= fc_nn(maze.size, maze.size, maze.size, 4)
    averaged_net= fc_nn(maze.size, maze.size, maze.size, 4)

optimizer_Q1 = optim.Adam(Q1_net.parameters(), lr=1e-4)
optimizer_Q2 = optim.Adam(Q2_net.parameters(), lr=1e-4)


In [ ]:
device = 'cuda:0'
batch_size = 24
gamma = 0.95

Q1_net.to(device)
Q2_net.to(device)
averaged_net.to(device)


**Define the epsilon profile and plot the resetting probability.**

In [ ]:
num_epochs = 20000

cutoff = 3000
epsilon = np.exp(-np.arange(num_epochs)/(cutoff))
epsilon[epsilon > epsilon[100*int(num_epochs/cutoff)]] = epsilon[100*int(num_epochs/cutoff)]
plt.plot(epsilon, color = 'orangered', ls = '--')
plt.xlabel('Epochs')
plt.ylabel('Epsilon')
plt.savefig('epsilon_profile.pdf', dpi = 300, bbox_inches = 'tight')
plt.show()


**Training the network.**

In [ ]:
loss_log = []
best_loss = 1e5
previous_running_loss = 1e5  # Initialize with a high value
running_loss = 0
loss_diff = 1e5
average_rewards_log = []
estop = "N/A"

for epoch in range(num_epochs):
    loss = 0
    counter = 0
    eps = epsilon[epoch]
    
    agent.isgameon = True
    _ = agent.env.reset(eps)
    
    while agent.isgameon:
        agent.make_a_move_doubleQ(Q1_net, Q2_net, eps,counter)
        counter += 1

        if len(agent.buffer) < buffer_start_size:
            continue
            
        # Sample from replay buffer
        batch = agent.buffer.sample(batch_size, device=device)
        
        # Decide which network to update
        if random.random() > 0.5:
            optimizer = optimizer_Q1
            loss_t = DoubleQloss(batch, Q1_net, Q2_net, gamma=gamma, device=device)
            Q1_net.zero_grad()
        else:
            optimizer = optimizer_Q2
            loss_t = DoubleQloss(batch, Q2_net, Q1_net, gamma=gamma, device=device)
            Q2_net.zero_grad()
        
        loss_t.backward()
        optimizer.step()
        
        loss += loss_t.item()
    
    if (agent.env.current_position == agent.env.goal).all():
        result = 'won'
    else:
        result = 'lost'
    
    if epoch % 1000 == 0:
        # Retrieve state dictionaries
        Q1_state_dict = Q1_net.state_dict()
        Q2_state_dict = Q2_net.state_dict()

        # Use dictionary comprehension to average the parameters
        averaged_state_dict = {key: (Q1_state_dict[key] + Q2_state_dict[key]) / 2.0 for key in Q1_state_dict}

        # Apply the averaged state dict to the new network
        averaged_net.load_state_dict(averaged_state_dict)

        agent.plot_policy_map(Q1_net, f'Results\\DoubleQ1_{model_type}_sol_epoch_{epoch}.pdf', [0, 0],f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")
        agent.plot_policy_map(Q2_net, f'Results\\DoubleQ2_{model_type}_sol_epoch_{epoch}.pdf', [0, 0],f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")
        agent.plot_policy_map(Q2_net, f'Results\\DoubleQ_averaged_{model_type}_sol_epoch_{epoch}.pdf', [0, 0],f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")
        agent.plot_Q_values(averaged_net, f'Results\\DoubleQ_{model_type}_sol_epoch_{epoch}_Q_values.pdf', [0, 0],f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")

    loss_log.append(loss)
    
    # Evaluate average loss every 50 epochs after 2000 epochs
    
    if epoch > 2000 and epoch % 50 == 0:
        running_loss = np.mean(loss_log[-50:])
        # Update best loss and save the model if there's an improvement
        if running_loss < best_loss:
            best_loss = running_loss
            torch.save(Q1_net.state_dict(), f"Models\\best_Q1_{model_type}.torch")
            torch.save(Q2_net.state_dict(), f"Models\\best_Q2_{model_type}.torch")

            # Retrieve state dictionaries
            Q1_state_dict = Q1_net.state_dict()
            Q2_state_dict = Q2_net.state_dict()

            # Use dictionary comprehension to average the parameters
            averaged_state_dict = {key: (Q1_state_dict[key] + Q2_state_dict[key]) / 2.0 for key in Q1_state_dict}

            # Apply the averaged state dict to the new network
            averaged_net.load_state_dict(averaged_state_dict)

            torch.save(averaged_net.state_dict(), "Models\\best_averaged_{model_type}.torch")
            
            estop = epoch

    if epoch % 20 == 0:
        # Calculate and log the average reward for this run
        counter =0
        agent.isgameon = True
        _ = agent.env.reset_to_starting_tile()

        while agent.isgameon:
            agent.make_a_move_doubleQ(Q1_net,Q2_net, eps,counter, testing = True)
            counter += 1

        average_reward = agent.final_total_reward / counter if counter else 0
        average_rewards_log.append(average_reward)
    
    print('Epoch', epoch, '(number of moves ' + str(counter) + ')')
    print('Game', result)
    print('[' + '#'*(100-int(100*(1 - epoch/num_epochs))) +
          ' '*int(100*(1 - epoch/num_epochs)) + ']')
    print('\t Average loss: ' + f'{loss:.5f}')
    print('\t Average Accumulated Reward: ' + f'{average_rewards_log[-1]:.5f}')
    if epoch > 2000:
        print('\t Best running average loss: ' + f'{best_loss:.5f}' + ', achieved at epoch', estop)
        print('\t Running loss: ' + f'{running_loss:.5f}')
    clear_output(wait=True)

In [ ]:
torch.save(averaged_net.state_dict(), "Models\\averaged_net.torch")

In [ ]:
epochs_with_reward = range(0, num_epochs, 20)[:len(average_rewards_log)]
plt.figure(figsize=(10, 6))
plt.plot(epochs_with_reward, average_rewards_log, linestyle='-', color='blue')
plt.title(f'Average Accumulated Reward vs Epoch (Double DQN - {model_type})')
plt.xlabel('Epoch')
plt.ylabel('Average Accumulated Reward')
plt.grid(True)
plt.show()

**Show the maze solution and the policy learnt.**

In [ ]:
averaged_net.eval()
agent.isgameon = True
agent.use_softmax = False
_ = agent.env.reset(0)
while agent.isgameon:
    agent.make_a_move(averaged_net, 0)
    agent.env.draw('')
    clear_output(wait = True)

In [ ]:
agent.plot_policy_map(averaged_net, 'Results\\DoubleQ_solution.pdf', [0,0], f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")
agent.plot_Q_values(averaged_net, 'Results\\DoubleQ_solution_Values.pdf', [0, 0],f'Gamma ({gamma}), Buffer Capacity ({buffer_capacity}), Batch Size ({batch_size}), Network Type ({model_type})',epoch,title="Double Q")
